In [1]:
import requests
import lxml
from lxml import etree

detail page

In [2]:
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.99 Safari/537.36"}

In [3]:
rep = requests.get(url='https://www.imdb.com/title/tt6823368', headers = headers)
html = etree.HTML(rep.content)

In [18]:
title = html.xpath('//body//div[@class="title_wrapper"]/h1/text()')[0]
grading = html.xpath('//body//div[@class="title_wrapper"]/div[@class="subtext"]/text()')[0].replace('\n','').strip()
release_date = int(html.xpath('//span[@id="titleYear"]/a/text()')[0])
rating = float(html.xpath('//div[@class="ratings_wrapper"]//span[@itemprop="ratingValue"]/text()')[0])
vote = int(html.xpath('//div[@class="ratings_wrapper"]//span[@itemprop="ratingCount"]/text()')[0].replace(',',''))

genres = []
language = []
country = []
director = []
writer = []
production_company = []
cast = []

for cell in html.xpath('//div[@id="titleDetails"]//div[@class="txt-block"]'):
    if len(cell.xpath('./h4/text()'))>0:
        if cell.xpath('./h4/text()')[0].replace(':','')=='Runtime':
            rumtime = int(cell.xpath('./time/text()')[0].replace('min','').strip())

        if cell.xpath('./h4/text()')[0].replace(':','')=='Country':
            country = '|'.join(cell.xpath('./a/text()')).replace('\n','').strip()

        if cell.xpath('./h4/text()')[0].replace(':','')=='Language':
            language = '|'.join(cell.xpath('./a/text()')).replace('\n','').strip()

        if cell.xpath('./h4/text()')[0].replace(':','')=='Also Known As':
            other_names = ' '.join(cell.xpath('./text()'))
            other_names = other_names.replace('\n','').strip()

        if cell.xpath('./h4/text()')[0].replace(':','')=='Production Co':
            production_company = '|'.join(cell.xpath('./a/text()')).replace('\n','').replace('| ','|').strip()

for cell in html.xpath('//div[@id="titleStoryLine"]//div[@class="see-more inline canwrap"]'):
    if len(cell.xpath('./h4/text()'))>0:
        if 'Genre' in cell.xpath('./h4/text()')[0].replace(':',''):
            genres = '|'.join(cell.xpath('./a/text()')).replace('\n','').strip()
        
        if 'Plot Keyword' in cell.xpath('./h4/text()')[0].replace(':',''):
            if len(cell.xpath('./nobr'))>0:
                print('having more keywords')
            else:
                key_words = '|'.join(cell.xpath('./a/span/text()')).replace('\n','').strip()
                
for cell in html.xpath('//div[@class="plot_summary "]//div[@class="credit_summary_item"]'):
    if len(cell.xpath('./h4/text()'))>0:
        if 'Director' in cell.xpath('./h4/text()')[0].replace(':',''):
            for a in cell.xpath('./a'):
                dire = a.xpath('./text()')[0].strip()
                if 'more' not in dire:
                    director.append(dire)
            director = '|'.join(director).replace('\n','').strip()

        if 'Writer' in cell.xpath('./h4/text()')[0].replace(':',''):
            for a in cell.xpath('./a'):
                writ = a.xpath('./text()')[0].strip()
                if 'more' not in writ:
                    writer.append(writ)
            writer = '|'.join(writer).replace('\n','').strip()

if len(html.xpath('//div[@id="titleCast"]//tr'))>1:
    cast = '|'.join(html.xpath('//div[@id="titleCast"]//tr/td[2]/a/text()')).replace('\n','').replace('| ', '|').strip()

having more keywords


In [19]:
print(title)
print(rumtime)
print(grading)
print(release_date)
print(rating)
print(vote)
print(genres)
print(language)
print(country)
print(other_names)
print(director)
print(writer)
print(production_company)
print(cast)

Glass 
129
PG-13
2019
6.7
184325
Drama| Sci-Fi| Thriller
English|Spanish
USA|China
Bộ Ba Quái Nhân
M. Night Shyamalan
M. Night Shyamalan
Universal Pictures|Blinding Edge Pictures|Blumhouse Productions
James McAvoy|Bruce Willis|Samuel L. Jackson|Anya Taylor-Joy|Sarah Paulson|Spencer Treat Clark|Charlayne Woodard|Luke Kirby|Adam David Thompson|M. Night Shyamalan|Shannon Destiny Ryan|Diana Silvers|Nina Wisner|Kyli Zion|Serge Didenko


key words page

In [144]:
rep = requests.get(url='https://www.imdb.com/title/tt6751668/keywords', headers = headers)
html = etree.HTML(rep.content)

In [149]:
key_words = '|'.join(html.xpath('//div[@id="keywords_content"]/table//td/@data-item-keyword')).strip()

plot summary page

In [3]:
rep = requests.get(url='https://www.imdb.com/title/tt3748528/plotsummary', headers = headers)
html = etree.HTML(rep.content)

In [4]:
synopsis = html.xpath('//ul[@id="plot-synopsis-content"]/li[@class="ipl-zebra-list__item"]')[0].xpath('string(.)')

In [41]:
li_list = html.xpath('//ul[@id="plot-summaries-content"]//li')
summary_list = []
summary = []
for li in li_list:
    if len(li.xpath('./div[@class="author-container"]'))>0:
        summary.append(li.xpath('.//p')[0].xpath('string(.)'))
        summary_text = ' '.join(summary).replace('\n','').strip()
        summary_list.append(summary_text)
        summary = []
    else:
        summary.append(li.xpath('.//p')[0].xpath('string(.)'))

In [42]:
for i in summary_list:
    print(i)

The daughter of an Imperial scientist joins the Rebel Alliance in a risky move to steal the Death Star plans.
All looks lost for the Rebellion against the Empire as they learn of the existence of a new super weapon, the Death Star. Once a possible weakness in its construction is uncovered, the Rebel Alliance must set out on a desperate mission to steal the plans for the Death Star. The future of the entire galaxy now rests upon its success.
As the Rebellion spreads its influence among people, the Galactic Empire builds a super weapon capable of destroying an entire planet. A group of rebels disobeys orders and take a shuttle to go on a mission to steal plans that hide secrets on how to destroy this weapon of mass destruction. In a short time period right before the events of A New Hope, we find ourselves in the middle of a galactic war between the Rebel Alliance and the Empire. With a desperate goal of retrieving the secret plans for the weapon of mass destruction: The Death Star, we f

User Rating Page

In [18]:
rep = requests.get(url='https://www.imdb.com/title/tt8946378/reviews?spoiler=hide&sort=helpfulnessScore&dir=desc&ratingFilter=0', headers = headers)
html = etree.HTML(rep.content)

In [23]:
def get_next_url(html, tid):
    data_key = '&ref_=undefined&paginationKey='+html.xpath('//div[@class="load-more-data"]/@data-key')[0]
    origin = 'http://www.imdb.com/title/%s/reviews/_ajax?sort=helpfulnessScore&dir=desc&spoiler=hide&ratingFilter=0'%tid
    return origin + data_key

In [20]:
def get_review_list(html):
    review_list = html.xpath('//div[@class="lister-list"]/div[@class="lister-item mode-detail imdb-user-review  collapsable"]')
    for review in review_list:
        score = review.xpath('.//span[@class="rating-other-user-rating"]/span[1]/text()')
        user = review.xpath('.//div[@class="display-name-date"]//a/text()')
        user_link = review.xpath('.//div[@class="display-name-date"]//a/@href')
        print('%s %s %s'%(user, score, user_link))

In [24]:
next_url = get_next_url(html, 'tt8946378')
get_review_list(html)

['pere-25366'] ['8'] ['/user/ur64640683/?ref_=tt_urv']
['kamalhasnaa'] ['9'] ['/user/ur85796457/?ref_=tt_urv']
['Coventry'] ['8'] ['/user/ur2020269/?ref_=tt_urv']
['hitsmenghwani'] ['10'] ['/user/ur60715768/?ref_=tt_urv']
['tcasavecchia'] ['8'] ['/user/ur8589011/?ref_=tt_urv']
['theawsomeadrian'] ['10'] ['/user/ur76122538/?ref_=tt_urv']
['johnryeh-49694'] ['9'] ['/user/ur110556035/?ref_=tt_urv']
['con-fan'] ['6'] ['/user/ur49815034/?ref_=tt_urv']
['bigmin'] ['4'] ['/user/ur100464064/?ref_=tt_urv']
['leeshep'] ['6'] ['/user/ur29467290/?ref_=tt_urv']
['Max_Simpson'] ['4'] ['/user/ur1110985/?ref_=tt_urv']
['backofthevan'] ['5'] ['/user/ur4947500/?ref_=tt_urv']
['iamlegend-07431'] ['8'] ['/user/ur74385628/?ref_=tt_urv']
['cli-13744'] ['10'] ['/user/ur106225835/?ref_=tt_urv']
['gabriel-milon'] ['7'] ['/user/ur37874086/?ref_=tt_urv']
['ialssaka'] ['8'] ['/user/ur110748227/?ref_=tt_urv']
['cgclip'] ['5'] ['/user/ur6148841/?ref_=tt_urv']
['btoews'] ['4'] ['/user/ur3400939/?ref_=tt_urv']
['Uniq

In [25]:
next_url

'http://www.imdb.com/title/tt8946378/reviews/_ajax?sort=helpfulnessScore&dir=desc&spoiler=hide&ratingFilter=0&ref_=undefined&paginationKey=g4wp7cbpqy5tkzih7svxzobqrxrm4bbhzfmxvlnomwklyczuf43o6ss6omyf5njddv4k5vtvxkpnwjb6fjifh2yghrjypsy'

In [26]:
rep = requests.get(url=next_url, headers = headers)
html = etree.HTML(rep.content)

In [27]:
next_url = get_next_url(html, 'tt8946378')
get_review_list(html)

['simonpcpearson'] ['3'] ['/user/ur23116197/?ref_=tt_urv']
['masonsaul'] ['8'] ['/user/ur94560739/?ref_=tt_urv']
['cruise01'] ['9'] ['/user/ur57691865/?ref_=tt_urv']
['Elcid_Asaei'] ['1'] ['/user/ur32740602/?ref_=tt_urv']
['adamk-2'] ['6'] ['/user/ur0693416/?ref_=tt_urv']
['marajademaster'] ['1'] ['/user/ur60380448/?ref_=tt_urv']
['michaeltodaro-99700'] ['1'] ['/user/ur107409402/?ref_=tt_urv']
['comake'] ['1'] ['/user/ur112735012/?ref_=tt_urv']
['tearitdownner'] ['1'] ['/user/ur89777878/?ref_=tt_urv']
['petercarlsson-92297'] ['1'] ['/user/ur82047401/?ref_=tt_urv']
['dmasursky'] ['6'] ['/user/ur2071359/?ref_=tt_urv']
['omidthegreat'] ['1'] ['/user/ur110645415/?ref_=tt_urv']
['kercherdillon'] ['1'] ['/user/ur101143658/?ref_=tt_urv']
['BrorsanW'] ['7'] ['/user/ur52912074/?ref_=tt_urv']
['amygwhite'] ['8'] ['/user/ur64755242/?ref_=tt_urv']
['edgy-827-486965'] ['1'] ['/user/ur23775976/?ref_=tt_urv']
['Kimberlyep'] ['1'] ['/user/ur6205640/?ref_=tt_urv']
['noahsofio'] ['2'] ['/user/ur83515282

In [28]:
next_url

'http://www.imdb.com/title/tt8946378/reviews/_ajax?sort=helpfulnessScore&dir=desc&spoiler=hide&ratingFilter=0&ref_=undefined&paginationKey=g4wp7dzeqizd6zaf7suhhmbwr3rm6bbhzfmxvlnomwklyczuf43o6ss6omzvznzjcr4k534avrerze2qnqwcgdiaokcflai'

In [29]:
rep = requests.get(url=next_url, headers = headers)
html = etree.HTML(rep.content)

In [30]:
next_url = get_next_url(html, 'tt8946378')
get_review_list(html)

['mrgoodvibrationsmichigan'] ['1'] ['/user/ur90124801/?ref_=tt_urv']
['james_r_webster'] ['1'] ['/user/ur56352258/?ref_=tt_urv']
['didkus'] ['3'] ['/user/ur18164100/?ref_=tt_urv']
['SerDarioTaranFulci'] ['1'] ['/user/ur42939296/?ref_=tt_urv']
['zeburock'] ['1'] ['/user/ur29554784/?ref_=tt_urv']
['ginorman-940-585477'] ['4'] ['/user/ur51166039/?ref_=tt_urv']
['cekadah'] ['3'] ['/user/ur35944897/?ref_=tt_urv']
['henrik-c-moller'] ['1'] ['/user/ur50105663/?ref_=tt_urv']
['Niebel'] ['2'] ['/user/ur3110434/?ref_=tt_urv']
['rachelwilcox-07516'] ['1'] ['/user/ur93646997/?ref_=tt_urv']
['sblrr'] ['5'] ['/user/ur57654421/?ref_=tt_urv']
['jmart-01179'] ['1'] ['/user/ur59987641/?ref_=tt_urv']
['nch-35770'] ['1'] ['/user/ur81639357/?ref_=tt_urv']
['johnnyfouraces'] ['1'] ['/user/ur90099948/?ref_=tt_urv']
['boozdog'] ['3'] ['/user/ur22362249/?ref_=tt_urv']
['charlottesperrin'] ['3'] ['/user/ur113376801/?ref_=tt_urv']
['iamtheonewhoknockss'] ['3'] ['/user/ur55287567/?ref_=tt_urv']
['philbojones'] ['

In [31]:
next_url

'http://www.imdb.com/title/tt8946378/reviews/_ajax?sort=helpfulnessScore&dir=desc&spoiler=hide&ratingFilter=0&ref_=undefined&paginationKey=g4wp7dzkr45to3yc76wxfnryqps4mbryy4hhzo5ziwr26fbyhvrl4ty4ouzvtnjmdfvndtuwkx3s3mpirtzyz5iyvd4zpvhz'

In [32]:
rep = requests.get(url=next_url, headers = headers)
html = etree.HTML(rep.content)
next_url = get_next_url(html, 'tt8946378')
get_review_list(html)

['clf-73998'] ['4'] ['/user/ur83311893/?ref_=tt_urv']
['athleticsman76'] ['1'] ['/user/ur26732005/?ref_=tt_urv']
['vittorio-13'] ['1'] ['/user/ur16456082/?ref_=tt_urv']
['Jared_Andrews'] ['8'] ['/user/ur53660798/?ref_=tt_urv']
['colossus34'] ['3'] ['/user/ur3979648/?ref_=tt_urv']
['gmaileatsyourlunch'] ['5'] ['/user/ur112194256/?ref_=tt_urv']
['pollak1000'] ['1'] ['/user/ur23069273/?ref_=tt_urv']
['morrigu-66655'] ['2'] ['/user/ur110516464/?ref_=tt_urv']
['hansvanopheusden'] ['1'] ['/user/ur55013298/?ref_=tt_urv']
['pjshaw1701'] ['1'] ['/user/ur51805580/?ref_=tt_urv']
['bios-96375'] ['1'] ['/user/ur107095145/?ref_=tt_urv']
['fccs23521'] ['5'] ['/user/ur84682955/?ref_=tt_urv']
['dlh-48964'] ['1'] ['/user/ur86507008/?ref_=tt_urv']
['donaldgordon797'] ['1'] ['/user/ur2099226/?ref_=tt_urv']
['hockey46'] ['5'] ['/user/ur33962673/?ref_=tt_urv']
['teksuremusic'] ['6'] ['/user/ur59998248/?ref_=tt_urv']
['alexa-tarasenko'] ['5'] ['/user/ur27828607/?ref_=tt_urv']
['markovd111'] ['6'] ['/user/ur5